Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Input, TimeDistributed, RepeatVector, Dense
from tensorflow.keras.layers import Bidirectional, \
    multiply, concatenate, Flatten, Activation, dot

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import keras

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae, rmse, r2 = mean_absolute_error, mean_squared_error, r2_score

# !pip install tensorflow_addons==0.16.1
import tensorflow_addons as tfa
from tensorflow_addons.optimizers import AdamW

c:\Users\bobby\anaconda3\envs\research\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\bobby\anaconda3\envs\research\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.war

In [2]:
def readdata(inputcsv, outputcsv):

    idx = pd.IndexSlice
    input_data2 = pd.read_csv(inputcsv, index_col=[0], header=[0,1])
    input_data2.index = pd.to_datetime(input_data2.index)
    input_data2.columns = input_data2.columns.set_levels(input_data2.columns.levels[0].astype('int64'), level=0)
    input_data2.columns = input_data2.columns.set_levels(input_data2.columns.levels[1].astype('string'), level=1)

    ground_truth2 = pd.read_csv(outputcsv, index_col=[0], header=[0,1])
    ground_truth2.index = pd.to_datetime(ground_truth2.index)
    ground_truth2.columns = ground_truth2.columns.set_levels(ground_truth2.columns.levels[0].astype('int64'), level=0)
    ground_truth2.columns = ground_truth2.columns.set_levels(ground_truth2.columns.levels[1].astype('string'), level=1)

    log_transform = lambda x: np.log10(x+1) if x.name[1] == 'tp' else x
    input_data2 = input_data2.apply(log_transform)
    ground_truth2 = ground_truth2.apply(log_transform)

    scaledx = MinMaxScaler()
    scaled_input = scaledx.fit_transform(input_data2.values)
    scaled_input_df = pd.DataFrame(scaled_input, index=input_data2.index, columns=input_data2.columns)

    scaledy = MinMaxScaler()
    scaled_ground = scaledy.fit_transform(ground_truth2.values)
    scaled_ground_df = pd.DataFrame(scaled_ground, index=ground_truth2.index, columns=ground_truth2.columns)

    frames = [scaled_input_df, scaled_ground_df]
    dataset = pd.concat(frames, axis=1)

    train_dataset = dataset.loc['2000-01-01':'2016-12-31']
    test_dataset = dataset.loc['2017-01-01':'2019-12-31']

    return train_dataset, test_dataset, scaledx, scaledy

train_dataset, test_dataset, scaledx, scaledy = readdata('input_data3.csv', 'ground_truth3.csv')

In [4]:
date_index = pd.date_range('2000-01-01','2016-12-22',freq='D') # changed to 2000-01-01 from 2000-01-10
break_index = [0]+[list(date_index).index(pd.to_datetime('%s-12-31'%year))+1
 for year in range(2000,2016)] + [len(date_index)]

def split_by_year(freq_year, break_index):
    # freq_year = 2
    end_index = 0
    tscv = []
    while True:
        start_index = end_index
        if start_index + freq_year >= len(break_index):
            break
        end_index = min(start_index+2*freq_year, len(break_index)-1)
        tscv.append((list(range(break_index[start_index],break_index[start_index+freq_year])),
                     list(range(break_index[start_index+freq_year],break_index[end_index]))))
    return tscv

freq_year = 2 # freq_year < total year/2  # 2000-2001 (2 years) test 2 years x4
tscv = split_by_year(freq_year, break_index)

def get_xy(series, time_step, n_feature):
    x = series.iloc[:,:-1].T.unstack(level=0).T.values.reshape(len(series),time_step,n_feature) # time_step will be 10
    y = pd.concat([series.iloc[:,-1].shift(-i) for i in range(time_step)], axis=1).dropna(axis=0, how='any').values
    y = y.reshape(y.shape[0],y.shape[1],1)
    x = x[:y.shape[0],:,:]
    return x, y

time_step, n_features = 10, 5
train_x, train_y = get_xy(train_dataset, time_step, n_features)
test_x, test_y = get_xy(test_dataset, time_step, n_features)

input_train = Input(shape=(train_x.shape[1], train_x.shape[2]))
output_train = Input(shape=(train_y.shape[1], train_y.shape[2]))

# gridsearchCV does not take in 3D shape as inputs, only 2D
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1]*train_x.shape[2])
train_y = train_y.reshape(train_y.shape[0], train_y.shape[1]*train_y.shape[2])

In [5]:
from tensorflow.python.ops.gen_math_ops import square
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import make_scorer

# Loss function with weights based on amplitude of y_true
import tensorflow as tf
from keras import backend as K

np.random.seed(42)
tf.random.set_seed(42)

def my_MSE_weighted2(y_true,y_pred):
    return K.mean(tf.multiply(tf.exp(tf.multiply(2.0, y_true)), tf.square(tf.subtract(y_pred, y_true))))

# def my_MSE_weighted2_5(y_true,y_pred):
#     return K.mean(tf.multiply(tf.exp(tf.multiply(2.5, y_true)), tf.square(tf.subtract(y_pred, y_true))))

# def my_MSE_weighted3(y_true,y_pred):
#     return K.mean(tf.multiply(tf.exp(tf.multiply(3.0, y_true)), tf.square(tf.subtract(y_pred, y_true))))

# scoring
def my_custom_eval_func(y_true, y_pred):
    # Remove 3D array warning
    if len(y_pred.shape) == 3:
        y_pred = y_pred.reshape(y_pred.shape[:-1])
    return rmse(y_true, y_pred, squared=False)

myenvEstimator  = make_scorer(my_custom_eval_func, greater_is_better=False)
#-------------------------------------------------------------------------------

from sklearn.base import BaseEstimator, ClassifierMixin
class mylstm(BaseEstimator, ClassifierMixin):
    def __init__(self, n_steps=10, n_features=5,
                 activation='relu', optimizer='adam',loss=my_MSE_weighted2,
                 lstm=48, dense=1, verbose=1,
                 epochs=20, batch_size=8):
                 #learning_rate=1e-3, #weight_decay=1e-5):

        # static parameters
        self.n_steps = n_steps
        self.n_features = n_features
        self.verbose = verbose

        # Parameters that can be optimized
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss
        self.lstm = lstm
        self.dense = dense
        self.epochs = epochs
        self.batch_size = batch_size

        #---------- luong attention ----------------
        encoder_stack_h, encoder_forward_h, encoder_forward_c, encoder_backward_h, encoder_backward_c = Bidirectional(LSTM(self.lstm, activation=self.activation,
                                                                                                                           return_state=True, return_sequences=True))(input_train)
        encoder_last_h = concatenate([encoder_forward_h, encoder_backward_h])
        encoder_last_c = concatenate([encoder_forward_c, encoder_backward_c])

        decoder_input = RepeatVector(output_train.shape[1])(encoder_last_h)
        decoder_stack_h = LSTM(self.lstm*2, activation=self.activation, return_state=False, return_sequences=True)(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
        attention = Activation('softmax')(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1])
        decoder_combined_context = concatenate([context, decoder_stack_h])
        out = TimeDistributed(Dense(output_train.shape[2]))(decoder_combined_context)
        self.model = Model(inputs=input_train, outputs=out)
        self.model.compile(optimizer=self.optimizer, loss=self.loss)

    def fit(self, X, y, **kw):
        X = X.reshape(X.shape[0], self.n_steps, self.n_features)

        # Control display output, If parameter `verbose` is given, it will be used.
        # If no `verbose` is given, the default value of the class is used.
        if 'verbose' in kw.keys():
            return self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, **kw)
        else:
            return self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose, **kw)


    def predict(self, X, **kw):
        X = X.reshape(X.shape[0], self.n_steps, self.n_features)

        if 'verbose' in kw.keys():
            return self.model.predict(X, **kw)
        else:
            return self.model.predict(X, verbose=self.verbose, **kw)

    def score(self, X, y, **kw):
        X = X.reshape(X.shape[0], self.n_steps, self.n_features)

        # Control display output
        if 'verbose' in kw.keys():
            return self.model.evaluate(X, y, **kw)
        else:
            return self.model.evaluate(X, y, verbose=self.verbose, **kw)

In [ ]:
# lr_schedule = tf.optimizers.schedules.ExponentialDecay(1e-3, 24, 0.95) # every 2 epochs
# wd_schedule = tf.optimizers.schedules.ExponentialDecay(1e-5, 24, 0.95)
# opt = AdamW(learning_rate=lr_schedule, weight_decay=lambda : None)
# opt.weight_decay = lambda : wd_schedule(opt.iterations)
opt = Adam(learning_rate=1e-4)

# parameters = {'activation':('relu', 'sigmoid', 'tanh'), 'lstm':[48,64,80], 'loss':(my_MSE_weighted, 'mse')} # Actual calculation
parameters = {'activation':('relu','tanh'), 'lstm':[32,48,64,80,100,128,144,160], 'loss':(my_MSE_weighted2, 'mse')} # Actual calculation
# parameters = {'activation':('relu','tanh')} # Actual calculation
# parameters = {} # original parameters

tscv = split_by_year(freq_year, break_index)
clf = GridSearchCV(mylstm(verbose=1, epochs=20, batch_size=8, optimizer=opt), parameters, cv=tscv, scoring=myenvEstimator)
# clf = GridSearchCV(mylstm(verbose=1, epochs=30, batch_size=64, optimizer=optimizer, parameters, cv=tscv, scoring=myenvEstimator))

clf.fit(train_x, train_y)

In [ ]:
results_df = pd.DataFrame(clf.cv_results_)
results_df['mean_test_score'] = results_df['mean_test_score'].abs()
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("kernel")
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

model_df = results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]
# model_df.to_csv('tunings_mse2_biadam.csv')

In [ ]:
clf.cv_results_

{'mean_fit_time': array([48.25649691, 35.09181929, 35.86329842, 35.25014049, 36.32160974,
        39.80626363, 40.09244198, 38.76100546, 39.58778954, 41.10896057,
        42.42620128, 43.13410336, 44.47147024, 44.06862843, 44.73675287,
        43.60585213, 56.02843034, 45.176467  , 56.78962451, 56.74436426,
        47.26439083, 57.05959105, 48.17155099, 48.85460126, 68.1179077 ,
        59.80541694, 59.39382249, 77.48450124, 59.83895636, 68.34578526,
        59.75040925, 59.13266689]),
 'mean_score_time': array([0.75094962, 0.68144286, 0.72104585, 0.8548755 , 0.70880604,
        0.78209698, 0.76868701, 0.73193061, 0.71389496, 0.75992006,
        0.73628891, 0.76536262, 1.02915251, 0.91230369, 0.82881784,
        0.77171475, 0.79725552, 0.93370324, 0.82907778, 0.78102207,
        0.84719265, 0.79590333, 0.85109013, 0.80885816, 0.82633185,
        0.74711382, 0.78590304, 0.94416595, 0.82948768, 0.88558722,
        0.95583731, 0.76971895]),
 'mean_test_score': array([-0.17140935, -0.16946